## Part 5

## Load

In [ ]:
%reload_ext google.cloud.bigquery

In [ ]:
%reload_ext google.cloud.bigquery

In [ ]:
import os
import pandas as pd
from google.cloud import bigquery
import google.auth
from google.cloud import bigquery

In [ ]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'aqueous-freedom-398208-cfe0d4e567f2.json'

In [ ]:
bigquery_client = bigquery.Client()

## Revenue Harian Perkota

In [ ]:
query1 = """

    SELECT
        o.order_date,
        r.city,
        SUM(f.price * o.quantity) AS daily_revenue
    FROM `aqueous-freedom-398208.OFD_Team_1.Order` o
    INNER JOIN`aqueous-freedom-398208.OFD_Team_1.Food` f ON o.fk_food_id = f.id_food
    INNER JOIN `aqueous-freedom-398208.OFD_Team_1.Restaurant` r ON f.fk_resto_id = r.id_restaurant
    GROUP BY order_date,city
    ORDER BY order_date,city;
"""

result1 = bigquery_client.query(query1)
df1 = result1.to_dataframe()
df1

,order_date,city,daily_revenue
0,2023-11-01,Surabaya,36000
1,2023-11-02,Bandung,36000
2,2023-11-03,Jakarta Barat,105000
3,2023-11-04,Bandung,36000
4,2023-11-05,Jakarta Barat,70000
...,...,...,...
87,2024-01-27,Jakarta Timur,60000
88,2024-01-28,Surabaya,50000
89,2024-01-29,Jakarta Barat,25000
90,2024-01-30,Jakarta Timur,90000


## 2. Customer segmentation


In [ ]:
query2 = """
drop table if exists`aqueous-freedom-398208.OFD_Team_1.costumer_segmentation`;
create table `aqueous-freedom-398208.OFD_Team_1.costumer_segmentation` AS
WITH tabel AS (
    SELECT
        id_customer,
        c.name,
        DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AS last_order_date,
        DATE_DIFF(o.order_date, c.registration_date, DAY) AS days_since_registration,
        COUNT(*) OVER (PARTITION BY c.name) AS total_transaction,
        SUM(f.price * o.quantity) OVER (PARTITION BY c.name) AS total_spending,
        ROW_NUMBER() OVER (PARTITION BY c.name ORDER BY o.order_date DESC) AS row_num,
        LAG(o.order_date) OVER (PARTITION BY c.name ORDER BY o.order_date) AS prev_order_date
    FROM aqueous-freedom-398208.OFD_Team_1.Customer c
    LEFT JOIN aqueous-freedom-398208.OFD_Team_1.Order o ON c.id_customer = o.fk_customer_id
    LEFT JOIN aqueous-freedom-398208.OFD_Team_1.Food f ON f.id_food = o.fk_food_id AND o.order_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
    WHERE c.registration_date <= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)
      AND o.order_date >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) -- Tambahkan kondisi untuk memfilter tanggal pesanan
)
SELECT
    id_customer,
    name,
    DATE_SUB(CURRENT_DATE(), INTERVAL 0 DAY) AS today_date,
    DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY) AS last_order,
    total_transaction,
    total_spending,
    CASE
        WHEN total_spending >= 500001 THEN 'Big Spender'
        WHEN total_transaction <= 1 THEN 'New'
        WHEN total_transaction <= 5 THEN 'Good'
        WHEN total_transaction <= 10 THEN 'Potential'
        ELSE 'Loyalist'
    END AS segmentation
FROM tabel
ORDER BY last_order_date DESC;


"""

result2 = bigquery_client.query(query2)
df2 = result2.to_dataframe()

df2

""


## Growth Harian Perkota

In [ ]:
query3 = """
drop table if exists `aqueous-freedom-398208.OFD_Team_1.growth_harian_perkota`;
create table `aqueous-freedom-398208.OFD_Team_1.growth_harian_perkota` AS
WITH DailyRevenue AS (
    SELECT
        r.city AS Kota,
        o.order_date AS Tanggal,
        SUM(o.quantity * f.price) AS Revenue,
        LAG(SUM(o.quantity * f.price), 1) OVER (PARTITION BY r.city ORDER BY o.order_date) AS Revenue_Hari_Kemarin,
        LAG(SUM(o.quantity * f.price), 2) OVER (PARTITION BY r.city ORDER BY o.order_date) AS Revenue_Dua_Hari_Sebelumnya
    FROM
        `aqueous-freedom-398208.OFD_Team_1.Order` o
    INNER JOIN
        `aqueous-freedom-398208.OFD_Team_1.Food` f ON o.fk_food_id = f.id_food
    INNER JOIN
        `aqueous-freedom-398208.OFD_Team_1.Customer` c ON o.fk_customer_id = c.id_customer
    INNER JOIN
        `aqueous-freedom-398208.OFD_Team_1.Restaurant` r ON f.fk_resto_id = r.id_restaurant
    GROUP BY
        r.city, o.order_date
)

SELECT
    Kota,
    Tanggal,
    COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya, 0) AS Revenue_Hari_Kemarin,
    Revenue,
    CASE
        WHEN COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya) IS NULL THEN NULL
        ELSE CONCAT(ROUND(((Revenue - COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya)) / NULLIF(COALESCE(Revenue_Hari_Kemarin, Revenue_Dua_Hari_Sebelumnya), 0)) * 100, 2), '%')
    END AS Growth_Rate_Percentage
FROM
    DailyRevenue
ORDER BY
    Kota, Tanggal;


"""

result3 = bigquery_client.query(query3)
df3 = result3.to_dataframe()
df3

""


## kontribusi harian per kota

In [ ]:
query4 = """
drop table if exists `aqueous-freedom-398208.OFD_Team_1.kontribusi_harian_perkota`;
CREATE TABLE `aqueous-freedom-398208.OFD_Team_1.kontribusi_harian_perkota` AS
WITH daily_total AS (
    SELECT
        order_date,
        SUM(f.price * o.quantity) AS total_revenue_all_cities
    FROM `aqueous-freedom-398208.OFD_Team_1.Order` o
    JOIN `aqueous-freedom-398208.OFD_Team_1.Food` f ON o.fk_food_id = f.id_food
    JOIN `aqueous-freedom-398208.OFD_Team_1.Restaurant` r ON f.fk_resto_id = r.id_restaurant
    GROUP BY
        order_date
)
SELECT
    o.order_date AS date,
    r.city,
    SUM(f.price * o.quantity) AS total_revenue,
    ROUND((SUM(f.price * o.quantity) / dt.total_revenue_all_cities) * 100, 2) AS contribution
FROM `aqueous-freedom-398208.OFD_Team_1.Order` o
JOIN `aqueous-freedom-398208.OFD_Team_1.Food` f ON o.fk_food_id = f.id_food
JOIN `aqueous-freedom-398208.OFD_Team_1.Restaurant` r ON f.fk_resto_id = r.id_restaurant
JOIN  daily_total dt ON o.order_date = dt.order_date
GROUP BY
    r.city, o.order_date, dt.total_revenue_all_cities
ORDER BY
    o.order_date, r.city;


    """

result4 = bigquery_client.query(query4)
df4 = result4.to_dataframe()

df4

""
